<a href="https://colab.research.google.com/github/zenon10/POC-OCR/blob/main/pdf_img_text_trans_fr_en_rg_02012023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Githup SetUp**

In [2]:
! git clone https://github.com/zenon10/POC-OCR

Cloning into 'POC-OCR'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 164 (delta 56), reused 21 (delta 21), pack-reused 70
Receiving objects: 100% (164/164), 21.36 MiB | 19.75 MiB/s, done.
Resolving deltas: 100% (74/74), done.


### Remove files from a folder

In [ ]:
# import glob
# files = glob.glob('/content/drive/MyDrive/Visa/text/it2/*')
# for f in files:
#     os.remove(f)

## Converting PDF to IMAGE

In [3]:
! pip install pypdfium2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 26.0 MB/s eta 0:00:00


### Importing Required Libraries

In [4]:
# Library to work with Operating Syatem
import os

# Library to create PDF, inspection, manupulation and rendering(turning into images)
import pypdfium2 as pdfium

# Regex lirary to manipulate string
import re

from PIL import Image

In [5]:
def pdf_to_img(input_file_path, output_folder_path):
    
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    input_file_name = os.path.split(input_file_path)[-1].replace('.pdf','')    
    print('Extracting text from {}'.format(input_file_name))
    
    # PdfDocument (supports file path strings, bytes, and byte buffers)
    pdf_info = pdfium.PdfDocument(input_file_path)
    
    # get the number of pages in the document
    n_pages = len(pdf_info)
    print(n_pages)
    
    page_indices = [i for i in range(n_pages)]
    
    # Render multiple pages concurrently(converting into images).
    # A bitmap is a type of memory organization or image file format used to store digital images.
    # PIL python imaging Library and it's the original library that enabled Python to deal with images.
    # Scale is the resolution of the image.
    
    renderer = pdf_info.render_to(pdfium.BitmapConv.pil_image, page_indices = page_indices, scale = 300/72)

    page_number = 1
    for images in renderer:
#         print(images.show())

        images.save(output_folder_path + "/" + str(page_number)  + '.jpeg')
        page_number +=1

- format like : it2_page-0001.jpeg
- https://stackoverflow.com/questions/37796598/how-to-sort-file-names-in-a-particular-order-using-python

In [6]:
# paths for function(pdf to img)
input_file_path = r"/content/POC-OCR/raw_data/f2.pdf"
output_folder_path ="/content/drive/MyDrive/Zenon/images/f2"

## calling the function
pdf_to_img(input_file_path, output_folder_path)

Extracting text from f2
4


## Extracting Img To text

In [11]:
!sudo apt install tesseract-ocr
!pip install pytesseract

#Explicitly for italian language
! sudo apt-get install tesseract-ocr-fra

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-fra
0 upgraded, 1 newly installed, 0 to remove and 27 not upgraded.
Need to get 528 kB of archives.
After this operation, 1,145 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-fra all 1:4.00~git30-7274cfa-1 [528 kB]
Fetched 528 kB in 4s (118 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> l

In [12]:
import os
import glob
import pandas as pd
import pytesseract
import shutil
import os
import random
try:
    from PIL import Image
except ImportError:
    import Image

In [13]:
def img_to_txt(image_path, language, output_folder_path):
    
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)
    
    try:
        path = glob.glob(image_path + "/*.jpeg")
        # print(path)
    except:
        print('image not in jpeg')
    
    
    df = pd.DataFrame(columns=['Image_path','output_folder_path', 'Language'])
        
    ## Blank string to concatenate all text string outputs from ocr for each image
    combined_text = ''
    
    for img in path:
        
        file_name = os.path.basename(img).replace('.jpeg','')
        print("File name:", file_name)
        
        ## Extracting text from image
        text = pytesseract.image_to_string(Image.open(img), lang=language)
        
        ## Concatenating text-string outputs from each image into one master text-string
        combined_text = ("** " + file_name + " **\n").join([combined_text, text])
        
        ## Creating separate text files for each image
        with open(output_folder_path + "/" + file_name +'.txt', "w",encoding="utf8") as f:
            f.write(text)
        
        ## Writing in a pandas dataframe
        df.loc[len(df.index)] = [img, output_folder_path + file_name +'.txt', language]
        
    ## Open a file with access mode 'a'
    file_object2 = open(output_folder_path + "/" + 'Combined_text.txt', 'a',encoding="utf8")
    file_object2.write(combined_text)
    file_object2.close()
    
    return df

In [14]:
# Paths for function(img to text)
image_path = '/content/drive/MyDrive/Zenon/images/f2'
language = 'fra'
output_folder_path = '/content/drive/MyDrive/Zenon/text/f2'

# Calling the function
df = img_to_txt(image_path, language, output_folder_path) ; df

File name: 1
File name: 2
File name: 3
File name: 4


,Image_path,output_folder_path,Language
0,/content/drive/MyDrive/Visa/images/f2/1.jpeg,/content/drive/MyDrive/Visa/text/f21.txt,fra
1,/content/drive/MyDrive/Visa/images/f2/2.jpeg,/content/drive/MyDrive/Visa/text/f22.txt,fra
2,/content/drive/MyDrive/Visa/images/f2/3.jpeg,/content/drive/MyDrive/Visa/text/f23.txt,fra
3,/content/drive/MyDrive/Visa/images/f2/4.jpeg,/content/drive/MyDrive/Visa/text/f24.txt,fra


In [ ]:
##GPU - Graphics processing unit
##TPU - Tensor processing unit

## Translation

In [ ]:
!pip install transformers[sentencepiece]

In [ ]:
from transformers import pipeline

qa = pipeline("text2text-generation", model='Helsinki-NLP/opus-mt-fr-en')


In [ ]:
!pip install sacremoses


In [ ]:
## loading txt file
d = '/content/drive/MyDrive/Zenon_POC_OCR/2_img_to_text/it_1'

In [ ]:
# def txt_to_trans(txt_path,output_trans_path):

#     if not os.path.exists(output_folder_path):
#         os.makedirs(output_folder_path)

#     path = glob.glob(txt_path + "/*.txt")
#     print(path)

#     df = pd.DataFrame(columns=['txt_path','output_trans_path'])

    


In [ ]:
## Reading text from .txt file and storing in a string
x = ''
with open(d) as f:
  for line in f:
    # print(line.strip())
    # print('STOP')
    x = x + line

In [ ]:
x

In [ ]:
## Splitting line/paragraph, translating one by one and saving to a master string
translated_master = ''

for i in range(x.count('\n\n')+1):
  print(i)
  # print(x.split('\n\n')[i])
  
  ## Splitting string by double line terminator
  y = x.split('\n\n')[i]
  
  ## Translating string
  trans = qa(y)
  # print("Translated is:", qa(y)[0]['generated_text'])
  
  translated_master = translated_master + '\n\n'+ trans[0]['generated_text']


In [ ]:
## Saving translated text to a .txt file
with open('/content/drive/MyDrive/translated.txt', 'w') as f:
  f.write(translated_master)
  f.close()